In [ ]:
import time
import undetected_chromedriver as uc
from bs4 import BeautifulSoup
import json

In [ ]:
def init_chromedriver(url):
    options = uc.ChromeOptions()
    options.headless = False
    options.user_data_dir="/home/jxt1n/.config/chromium/"
    driver = uc.Chrome(options=options)
    driver.get(url)
    return driver

In [ ]:
def load_json(filename):
    with open(filename, "r") as json_file:
        return json.load(json_file)


def dump_json(filename, data):
    with open(filename, "w") as json_file:
        json.dump(data, json_file)

In [ ]:
driver=init_chromedriver("https://www.facebook.com")

# Login into facebook

In [ ]:
driver.get("https://mbasic.facebook.com/me/friends")

In [ ]:
def click_see_more_friends():
    driver.find_element_by_id("m_more_friends").find_element_by_tag_name('a').click()

In [ ]:
def get_BirthDate(profile_url):
    if "?id=" in profile_url:
        driver.get(profile_url+"&v=info")
    else:
        driver.get(profile_url+"/about")
    soup=BeautifulSoup(driver.page_source,'html.parser')
    search_res=soup.find('td',text="Date of birth")
    if search_res:
        print("Birthdate is visible")
        dob=soup.find_all('td',text="Date of birth")[0].find_parent().find_all('td')[1].text
        print(dob)
        return dob
    else:
        print("Birthdate is hidden")
        return None

In [ ]:
def scrape_cur_page():
    tags=driver.find_elements_by_tag_name('tbody')
    for tag in tags[1:-2]:
        data=tag.find_element_by_tag_name('a')
        name=data.text
        profile_link=data.get_attribute('href')
        print(data.text)
        print(data.get_attribute('href'))
        if "hovercard" in profile_link:
            continue
        profile_link=profile_link.split('fref=fr_tab')[0][:-1]
        friend2url[name]=profile_link

In [ ]:
friend2url={}

In [ ]:
# code block to scrape all complete friend_list
def generate_friend2url():
    pg_count=1
    while True:
        print(pg_count)
        scrape_cur_page()
        try:
            click_see_more_friends()
            pg_count+=1
        except Exception as e:
            print("Reached End")
            break

In [ ]:
generate_friend2url()

In [ ]:
friend2url

In [ ]:
del(friend2url['Find Friends'])

In [ ]:
# dump friend2url to json to save progress

In [ ]:
dump_json("friend2url.json",friend2url)

In [ ]:
# can load friend2url directly now

In [ ]:
friend2url=load_json("friend2url.json")

In [ ]:
print(len(friend2url))

In [ ]:
friend2url

In [ ]:
def break_to_feel_real(): # not sure if needed anymore
    driver.get("https://www.jxtin.me")
    time.sleep(3)

In [ ]:
# gets ratelimited do not use
def generate_friend2bday():
    count=0
    for friend,url in friend2url.items():
        count+=1
        bday=get_BirthDate(url)
        time.sleep(0.5)
        if bday is not None:
            friend2bday[friend]=bday
        if count%30==0:
            break_to_feel_real()

In [ ]:
def get_BirthDate_old(profile_url): # can be used
    profile_url=profile_url.replace("mbasic","www")
    if "?id=" in profile_url:
        driver.get(profile_url+"&sk=about_contact_and_basic_info")
    else:
        driver.get(profile_url+"/about_contact_and_basic_info")
    soup=BeautifulSoup(driver.page_source,'html.parser')
    year_str=soup.find_all("div",string="Birth year")
    date_str=soup.find_all("div",string="Birth date")
    if (date_str): # need to include partial cases by changing and to or
        print("DOB-Exists")
        bday=date_str[0].find_parent().text.split('B')[0]
        print(bday)
        return bday
    else:
        print("DOB no exist")

In [ ]:
def generate_friend2bday_old(): # yes use this
    count=0
    for friend,url in list(friend2url.items())[:]:
        count+=1
        bday=get_BirthDate_old(url)
        time.sleep(0.5)
        if bday is not None:
            print("should append")
            friend2bday[friend]=bday
            print(friend2bday)
        if count%30==0:
            break_to_feel_real()

In [ ]:
friend2bday={}
generate_friend2bday_old()

In [ ]:
friend2bday

In [73]:
len(friend2bday)

1233

In [ ]:
dump_json("friend2bday.json",friend2bday)